In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
from tools.models import ResNet18

model = ResNet18()

model.load_state_dict(torch.load('best_modelResNet18_dni_adjusted.pth', map_location=torch.device('cpu')))

model.to(device)

model.eval() 

In [ ]:
from PIL import Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

img = Image.open('DNI-Invalidos/IMG_20241023_122058232.jpg').convert('RGB')
img = transform(img)
img = img.unsqueeze(0).to(device)

In [ ]:
with torch.no_grad():
    output = model(img)
    probability = torch.sigmoid(output).item()
    prediction = "valida" if probability > 0.5 else "invalida"

print(f'La imagen es {prediction} con una probabilidad de {probability:.10f}')